<a href="https://colab.research.google.com/github/sanaa-04/Finetuning_Pretrained_Model_of_HuggingFace/blob/main/FinetuningPretrainedModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install peft
!pip install accelerate
!pip install bitsandBytes
!pip install transformers
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 13.4 MB/s eta 0:00:00


In [ ]:
!pip install GPUtil

  Preparing metadata (setup.py) ... done
  Created wheel for GPUtil: filename=GPUtil-1.4.0-py3-none-any.whl size=7392 sha256=f4a0dfff0458cfc9de53b4f632a17fd488fd8c0e29f2713754a9c3dc4af45bcf
  Stored in directory: /root/.cache/pip/wheels/92/a8/b7/d8a067c31a74de9ca252bbe53dea5f896faabd25d55f541037
Successfully built GPUtil


In [ ]:
import torch
import GPUtil
import os

GPUtil.showUtilization()

if torch.cuda.is_available():
    print("GPU is available")
else:
    print("GPU is not available, using CPU instead")

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

| ID | GPU | MEM |
------------------
|  0 |  0% |  0% |
GPU is available


In [ ]:
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, LlamaTokenizer
from huggingface_hub import notebook_login
from datasets import load_dataset
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model

if "COLAB_GPU" in os.environ:
  from google.colab import output
  output.enable_custom_widget_manager()

In [ ]:
if "COLAB_GPU" in os.environ:
  !huggingface-cli login
else:
  notebook_login()

⚠️  Warning: 'huggingface-cli login' is deprecated. Use 'hf auth login' instead.

    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `hf auth whoami` to get more information or `hf auth logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add t

In [ ]:
base_model_id = "meta-llama/Llama-2-7b-chat-hf"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(base_model_id, quantization_config=bnb_config)

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [ ]:
from datasets import load_dataset


dataset = load_dataset("AlignmentLab-AI/agentcode")

# Display the first few examples of the training split
# display(dataset['train'].head())

In [ ]:
display(dataset['train'][:5])

{'INSTRUCTION': ['Could you provide an explanation on Python dictionaries, including how they are different from lists, how they are defined, how keys and values work, and show a practical example of a dictionary in code?',
  'Please provide an explanation of the dictionary data type in Python, how it differs from lists, how to create one, an example of its creation, and how to access its stored data.',
  "What is the size of your cat as represented in your dictionary 'myCat', how would you describe your cat's fur color using concatenation, and can you explain the fundamental differences between lists and dictionaries with an example for each including key-value pairs?",
  'Please provide a comprehensive introduction to Python dictionaries, explaining what they are, how to access values, using integers as keys, the difference between dictionaries and lists, and why they are useful for storing data.',
  "Please compare the two lists named 'spam' and 'bacon'. Then, can you also compare t

In [ ]:
tokenizer = LlamaTokenizer.from_pretrained(base_model_id, use_fast=False, trust_remote_code=True, add_eos_token=True)

if tokenizer.pad_token is None:
  tokenizer.add_special_tokens({'pad_token': tokenizer.eos_token})

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

In [ ]:
tokenized_train_dataset = []
for phrase in dataset['train']:
  tokenized_train_dataset.append(tokenizer(phrase["INSTRUCTION"]))

In [ ]:
tokenized_train_dataset[1]

{'input_ids': [1, 3529, 3867, 385, 8252, 310, 278, 8600, 848, 1134, 297, 5132, 29892, 920, 372, 2923, 414, 515, 8857, 29892, 920, 304, 1653, 697, 29892, 385, 1342, 310, 967, 11265, 29892, 322, 920, 304, 2130, 967, 6087, 848, 29889, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
tokenized_train_dataset[2]

{'input_ids': [1, 1724, 338, 278, 2159, 310, 596, 6635, 408, 9875, 297, 596, 8600, 525, 1357, 9694, 742, 920, 723, 366, 8453, 596, 6635, 29915, 29879, 3261, 2927, 773, 16125, 362, 29892, 322, 508, 366, 5649, 278, 15281, 12651, 1546, 8857, 322, 21503, 4314, 411, 385, 1342, 363, 1269, 3704, 1820, 29899, 1767, 11000, 29973, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
tokenizer.eos_token

'</s>'

In [ ]:
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

config = LoraConfig(
    r=8,
    lora_alpha=64,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    bias="none",
    lora_dropout=0.05,
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)

In [ ]:
trainer = transformers.Trainer(
    model=model,
    train_dataset=tokenized_train_dataset,
    args=transformers.TrainingArguments(
        output_dir="./finetunedModel",
        per_device_train_batch_size=2,
        gradient_accumulation_steps=2,
        num_train_epochs=3,
        learning_rate=1e-4,
        max_steps=20,
        bf16=False,
        optim="paged_adamw_8bit",
        logging_dir="./log",
        save_strategy="epoch",
        save_steps=50,
        logging_steps=10

),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache=False
trainer.train()

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: sanamansoori705 (sanamansoori705-woolf) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:929: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
10,2.629900
20,2.133700


TrainOutput(global_step=20, training_loss=2.381787967681885, metrics={'train_runtime': 702.8105, 'train_samples_per_second': 0.114, 'train_steps_per_second': 0.028, 'total_flos': 528145349394432.0, 'train_loss': 2.381787967681885, 'epoch': 0.00036183377356442453})

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import BitsAndBytesConfig, LlamaTokenizer
from peft import PeftModel

base_model_id = "meta-llama/Llama-2-7b-chat-hf"

nf4Config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = LlamaTokenizer.from_pretrained(base_model_id, use_fast=False, trust_remote_code=True, add_eos_token=True)

base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    quantization_config=nf4Config,
    device_map="auto",
    trust_remote_code=True,
    use_auth_token=True
  )


/usr/local/lib/python3.12/dist-packages/transformers/models/auto/auto_factory.py:492: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
tokenizer = LlamaTokenizer.from_pretrained(base_model_id, use_fast=False, trust_remote_code=True, add_eos_token=True

                              )

modelFinetuned = PeftModel.from_pretrained(base_model, "finetunedModel/checkpoint-20")

In [ ]:
user_question = "Please provide a brief explanation of how to create and handle PDF files with PyPDF2 in Python, including its capabilities and limitations."

eval_prompt = f"Question: {user_question} Just answer this question accurately and concisely.\n"

promptTokenized = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

modelFinetuned.eval()

with torch.no_grad():
  print(tokenizer.decode(modelFinetuned.generate(**promptTokenized, max_new_tokens=1024)[0], skip_special_tokens=True))
  torch.cuda.empty_cache()

Question: Please provide a brief explanation of how to create and handle PDF files with PyPDF2 in Python, including its capabilities and limitations. Just answer this question accurately and concisely.
: Sure, here's a brief explanation of how to create and handle PDF files with PyPDF2 in Python, including its capabilities and limitations:

PyPDF2 is a Python library for reading and writing PDF files. It provides a simple and easy-to-use interface for creating and manipulating PDF documents. Here are some basic operations you can perform with PyPDF2:

Creating a PDF file:

To create a new PDF file using PyPDF2, you can use the `open()` function and pass it a file name as the first argument, followed by the content of the PDF file as a string. Here's an example:
```
import PyPDF2

# Create a new PDF file
with PyPDF2.open('example.pdf', 'w') as pdf:
    pdf.add_page('Hello, World!')

# Save the file
pdf.save()
```
In this example, we create a new PDF file named `example.pdf` and add a si

In [ ]:
import gradio as gr
import torch

# Define a function that uses the finetuned model to generate text
def generate_response(user_input):
    eval_prompt = f"Question: {user_input} Just answer this question accurately and concisely.\n"
    promptTokenized = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

    modelFinetuned.eval()
    with torch.no_grad():
        output_tokens = modelFinetuned.generate(**promptTokenized, max_new_tokens=1024)
        response = tokenizer.decode(output_tokens[0], skip_special_tokens=True)

    # Clean up the response to remove the original prompt
    # Find the index where the model's generated text starts
    prompt_end_index = response.find("Just answer this question accurately and concisely.\n")
    if prompt_end_index != -1:
        # Add the length of the prompt part we want to keep
        response_start_index = prompt_end_index + len("Just answer this question accurately and concisely.\n")
        # Find the start of the model's actual response after the prompt
        response_start = response[response_start_index:].strip()
        # Assuming the model output starts with ":" as seen in previous runs
        if response_start.startswith(":"):
          response = response_start[1:].strip() # Remove the leading ":" and any extra whitespace
        else:
          response = response_start # Keep the response as is if it doesn't start with ":"
    else:
        # If the prompt structure isn't found, return the full response
        response = response.strip()


    torch.cuda.empty_cache()
    return response

# Create the Gradio interface
iface = gr.Interface(
    fn=generate_response,
    inputs=gr.Textbox(lines=2, placeholder="Enter your question here..."),
    outputs="text",
    title="Finetuned Llama-2 Chatbot"
)

# Launch the interface
iface.launch(debug=True)

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://ed1eb9c8d9b939c9dd.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
